# <center> Japan Influenza Trends 2024-2025: A Data Analysis </center>
### Nicole Rouleau
### October 2025

## Introduction

This project takes a look at Japan from May 2024 to May 2025 and the impact of influenza on the country as a whole. The news reported influenza had a high impact on the country this last season, according to outlets like [Kyodo News](https://english.kyodonews.net/articles/-/52360?phrase=japan&words=) and [Global Newswire](https://www.globenewswire.com/news-release/2025/02/25/3031795/0/en/Flu-Crisis-Escalates-in-Asia-European-Wellness-Biomedical-Group-Advocates-Cutting-Edge-Regenerative-Medicine-for-Immunity-Boost.html). Looking at the data, we can see the effects of the disease on the country and just how devastating the numbers were.

## Data
The data for this project was gathered primarily from the Japan Institute for Health Security, 'The Infectious Disease Website', as well as the World Health Organization. The collected data includes much of the 2024-2025 influenza season with: reports by age and sex, deaths by age, hospitalizations by age, detection of types of viruses, detection by prefecture, and detection by virus with drug resistance. Data was scraped and cleaned in Excel prior to import.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
import seaborn as sns
import numpy as np
import json
import urllib.request
import plotly.express as px
import plotly.io as pio

# Load workbook
xls = pd.ExcelFile('Japan_Influenza_2425.xlsx')
sheet_names = xls.sheet_names
dfs = {sheet: xls.parse(sheet) for sheet in sheet_names}

print(dfs.keys())

In [ ]:
# Assign Each Sheet to its own variable for easy DataFrame use
detectresist_df = dfs['Detection_AV_DR_Inf_2425']
detectprefect_df = dfs['Detection_H1N1_mH275Y_byp_2425']
situation_df = dfs['Situation_Updates_2425']
detectsubtype_df = dfs['Detections_by_Subtype_2425']
detectweek_df = dfs['Detection_H1N1_mH275Y_byw_2425']
cases_df = dfs['ReportedByAge']
hosp_df = dfs['HospByAge']
death_df = dfs['Deaths by Age']

# Data Overview
for name, df in dfs.items():
    print(f"\n=== {name} ===")
    print(f"Rows: {len(df)}, Columns: {list(df.columns)}")

## Analysis
### Descriptive Statistics

In [ ]:
# -- Total Influenza Cases --
total_cases = cases_df['Count'].sum()
print(f"Total influenza cases (overall): {total_cases:,}")

# -- Cases Seasonally --
## Each record in the dataset represents influenza reports aggregated weekly. Because the time format follows a `YYYY-WW` structure, I extracted the year and week values for temporal analysis.
cases_df[['Year', 'Week']] = cases_df['Year-week '].astype(str).str.split('-', expand=True)
cases_df['Year'] = cases_df['Year'].astype(int)
cases_df['Week'] = cases_df['Week'].astype(int)

## Combine year and Week into a single numeric order
### To align the 2024–2025 influenza season into a single continuous timeline, I reindexed calendar weeks into a new variable `SeasonWeek`. This vectorized approach uses pandas masks for performance and clarity, avoiding row-level iteration while preserving readability.
cases_df['SeasonWeek'] = pd.NA
mask_24 = cases_df['Year'] == 2024
mask_25 = cases_df['Year'] == 2025
cases_df.loc[mask_24, 'SeasonWeek'] = cases_df.loc[mask_24, 'Week'] - 34
cases_df.loc[mask_25, 'SeasonWeek'] = 18 + cases_df.loc[mask_25, 'Week']
cases_df['SeasonWeek'] = cases_df['SeasonWeek'].astype('Int64')

season = cases_df.dropna(subset=['SeasonWeek']).copy()
cases_by_season_week = season.groupby('SeasonWeek', as_index=False)['Count'].sum()

## Peak SeasonWeek
peak_row = cases_by_season_week.loc[cases_by_season_week['Count'].idxmax()]
peak_week = int(peak_row['SeasonWeek'])
peak_val = int(peak_row['Count'])

plt.figure(figsize=(10,6))
plt.plot(cases_by_season_week['SeasonWeek'], cases_by_season_week['Count'], marker='o')
plt.title('Japan Influenza Cases: 2024–2025 Season (W35–W19)')
plt.xlabel('Season Week (1 = 2024 W35)')
plt.ylabel('Reported Cases')
plt.show()

print(f"Peak influenza activity occurred in Season Week {peak_week} with {peak_val:,} reported cases.")

# -- Cases by Week --
cases_by_week = cases_df.groupby(['Year', 'Week'], as_index=False)['Count'].sum()

# -- Case Distribution --
mean_cases = cases_df['Count'].mean()
median_cases = cases_df['Count'].median()
std_cases = cases_df['Count'].std()

print(f"Average weekly cases: {mean_cases:,.0f}")
print(f"Median weekly cases: {median_cases:,.0f}")
print(f"Standard deviation (spread): {std_cases:,.0f}")

# -- Cases by Age & Sex --
cases_by_age_sex = cases_df.groupby(['Label', 'Sex'], as_index=False)['Count'].sum()

plt.figure(figsize=(10,6))
sns.barplot(data=cases_by_age_sex, x='Label', y='Count', hue='Sex', palette=['#0072B2', '#E69F00'], edgecolor='black')

plt.title('Japan Influenza Cases by Age Group and Sex (2024–2025)')
plt.xlabel('Age Group')
plt.ylabel('Reported Cases')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Sex')
plt.tight_layout()
plt.show()

## Overall Peak Age Group x Sex
peak_overall = cases_by_age_sex.loc[cases_by_age_sex['Count'].idxmax()]
print(
    f"Overall peak group: {peak_overall['Label']} ({peak_overall['Sex']}) "
    f"with {int(peak_overall['Count']):,} cases.")

## Peak Week per Age Group x Sex
by_age_sex_week = (
    cases_df.dropna(subset=['SeasonWeek'])
            .groupby(['Label', 'Sex', 'SeasonWeek'], as_index=False)['Count']
            .sum())

idx = by_age_sex_week.groupby(['Label', 'Sex'])['Count'].idxmax()
peaks_by_age_sex = by_age_sex_week.loc[idx].sort_values('Count', ascending=False)

peaks_by_age_sex = peaks_by_age_sex.rename(
    columns={'SeasonWeek': 'Peak_SeasonWeek', 'Count': 'Peak_Count'})

print("Peak week per age group × sex (sorted by peak size):")
display(peaks_by_age_sex.head(15))

## Peak Sex 
cases_by_sex = cases_df.groupby('Sex', as_index=False)['Count'].sum().sort_values('Count', ascending=False)
peak_sex = cases_by_sex.iloc[0]

plt.figure(figsize=(6,4))
sns.barplot(data=cases_by_sex, x='Sex', y='Count', color='#0072B2')
plt.title('Total Influenza Cases by Sex (2024–2025)')
plt.ylabel('Reported Cases')
plt.xlabel('Sex')
plt.tight_layout()
plt.show()

## Peak Age Group
cases_by_age = cases_df.groupby('Label', as_index=False)['Count'].sum().sort_values('Count', ascending=False)
peak_age = cases_by_age.iloc[0]

plt.figure(figsize=(6,4))
sns.barplot(data=cases_by_age, x='Label', y='Count', color='#0072B2')
plt.title('Total Influenza Cases by Age Group (2024–2025)')
plt.ylabel('Reported Cases')
plt.xlabel('Age Group')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print(f"Peak age group: {peak_age['Label']} with {int(peak_age['Count']):,} total reported cases.")
print(f"Peak sex: {peak_sex['Sex']} with {int(peak_sex['Count']):,} total reported cases.")

### Trend Analysis

In the descriptive statistics section, we established the overall shape and timing of influenza activity for the 2024–2025 season using the unified `SeasonWeek` index.  
This section extends that analysis by examining how cases evolve week-to-week, identifying short-term momentum, and highlighting age-specific temporal patterns.

In [ ]:
# Data Prep
season_trend = cases_by_season_week.copy()
season_trend = season_trend.sort_values('SeasonWeek')

# 3-week Rolling Average
season_trend['Rolling3'] = season_trend['Count'].rolling(window=3, center=True).mean()

plt.figure(figsize=(10,6))
plt.plot(season_trend['SeasonWeek'], season_trend['Count'], alpha=0.4, label='Weekly Cases')
plt.plot(season_trend['SeasonWeek'], season_trend['Rolling3'], color='#0072B2', linewidth=2.5, label='3-Week Rolling Avg')

plt.scatter(peak_week, peak_val, color='red', s=100, zorder=5, label='Peak Week')
plt.text(
    peak_week + 0.3,
    peak_val,
    f"Peak: SeasonWeek {int(peak_week)}\n{int(peak_val):,} cases",
    color='red',
    fontsize=10
)

plt.title('Influenza Trend (2024–2025 Season) — 3-Week Rolling Average')
plt.xlabel('Season Week')
plt.ylabel('Reported Cases')
plt.legend()
plt.grid(alpha=0.4)
plt.tight_layout()
plt.show()

print(
    '''
    The rolling-average curve smooths natural week-to-week noise and reveals the underlying trend.  
    The seasonal peak occurred at SeasonWeek 18, which corresponds to ISO Week 52 of 2024 (late December).  
    This aligns with expected influenza seasonality in Japan: a sharp early-winter surge followed by a slower decline through January.
    '''
    )

# Age Time-Series
age_week = (
    cases_df
    .dropna(subset=['SeasonWeek'])
    .groupby(['Label', 'SeasonWeek'], as_index=False)['Count']
    .sum()
)

g = sns.FacetGrid(
    age_week, 
    col="Label", 
    col_wrap=4,        
    height=2.2, 
    sharey=False       
)

g.map_dataframe(
    sns.lineplot,
    x="SeasonWeek",
    y="Count",
    color="#0072B2"
)

g.set_titles(col_template="{col_name}")
g.set_axis_labels("Season Week", "Cases")
plt.tight_layout()
plt.show()

print(
    '''
    To avoid overplotting 20 age groups on a single line chart, I visualized the 2024–2025 influenza season as a grid of small multiples (one panel per age group). 
    Each subplot shows weekly case counts over `SeasonWeek`, starting at 2024 Week 35 and ending at 2025 Week 19.
    The age group with the highest peak is evidently the 10-14 year old range, which aligns with expectations for morbidity in school-aged children.
    '''
    )

The 2024–2025 influenza season showed a sharp rise leading into the peak at `SeasonWeek` 18, followed by a gradual decline.
Rolling averages revealed a clearer trend shape by smoothing weekly noise.
Age-specific temporal analysis showed distinct timing differences among demographic groups, highlighting how transmission dynamics vary across populations.

These trend insights contextualize the descriptive findings and prepare the foundation for severity and outcome analysis (hospitalizations and deaths) in the next section.

### Severity Analysis

In this section, I quantify the severity of the 2024–2025 influenza season in Japan using hospitalizations and deaths by age group. Because the case dataset uses fine-grained age bands (e.g., “0–5 months”, “6–11 months”, “1 yr”), while the hospitalization and death tables use broader age bins (e.g., 0–1, 1–9, 10–19, …), I first align case counts to the
same age bins as the severity data. Severity is then expressed as: hospitalizations per 10,000 cases and deaths per 100,000 cases.

In [ ]:
# Clean column names
cases_df.columns = cases_df.columns.str.strip()
hosp_df.columns = hosp_df.columns.str.strip()
death_df.columns = death_df.columns.str.strip()

# Ensure numeric case counts and ages
cases_df['Count'] = pd.to_numeric(cases_df['Count'], errors='coerce').fillna(0)
cases_df['Start Age (years)'] = pd.to_numeric(cases_df['Start Age (years)'], errors='coerce')
cases_df['End Age (years)']   = pd.to_numeric(cases_df['End Age (years)'], errors='coerce')

hosp_df['Hospitilizations'] = pd.to_numeric(hosp_df['Hospitilizations'], errors='coerce').fillna(0)
death_df['Deaths']          = pd.to_numeric(death_df['Deaths'], errors='coerce').fillna(0)

hosp_df['Start Age'] = pd.to_numeric(hosp_df['Start Age'], errors='coerce')
hosp_df['End Age']   = pd.to_numeric(hosp_df['End Age'], errors='coerce')
death_df['Start Age'] = pd.to_numeric(death_df['Start Age'], errors='coerce')
death_df['End Age']   = pd.to_numeric(death_df['End Age'], errors='coerce')

# Aggregate Age
age_bins = (
    hosp_df[['Start Age', 'End Age']]
    .drop_duplicates()
    .sort_values(['Start Age', 'End Age'])
    .reset_index(drop=True)
)

rows = []
for _, bin_row in age_bins.iterrows():
    bstart = bin_row['Start Age']
    bend   = bin_row['End Age']
    
    if np.isnan(bend):
        # open-ended bin, e.g. 80+
        mask = cases_df['Start Age (years)'] >= bstart
    else:
        # include all case age bands fully inside the bin
        mask = (
            (cases_df['Start Age (years)'] >= bstart) &
            (cases_df['End Age (years)']   <= bend)
        )
    
    bin_cases = cases_df.loc[mask, 'Count'].sum()
    rows.append({'Start Age': bstart, 'End Age': bend, 'Cases': bin_cases})

cases_binned = pd.DataFrame(rows)
cases_binned

# Aggregate hosp/ deaths
hosp_binned = (
    hosp_df.groupby(['Start Age','End Age'], as_index=False)['Hospitilizations']
           .sum()
)

death_binned = (
    death_df.groupby(['Start Age','End Age'], as_index=False)['Deaths']
            .sum()
)

severity_binned = (
    cases_binned
    .merge(hosp_binned, on=['Start Age','End Age'], how='left')
    .merge(death_binned, on=['Start Age','End Age'], how='left')
)

severity_binned[['Hospitilizations','Deaths']] = (
    severity_binned[['Hospitilizations','Deaths']].fillna(0)
)
severity_binned

# Severity Rates
severity_binned['HospRate_per10k']   = (
    severity_binned['Hospitilizations'] / severity_binned['Cases'] * 10000
)
severity_binned['DeathRate_per100k'] = (
    severity_binned['Deaths'] / severity_binned['Cases'] * 100000
)

## Age Bin
def format_age_bin(row):
    start = int(row['Start Age'])
    if np.isnan(row['End Age']):
        return f"{start}+"
    else:
        end = int(row['End Age'])
        return f"{start}-{end}"

severity_binned['AgeBin'] = severity_binned.apply(format_age_bin, axis=1)

severity_binned[['AgeBin','Cases','Hospitilizations','Deaths',
                 'HospRate_per10k','DeathRate_per100k']]

# Plot
severity_binned = severity_binned.sort_values('Start Age')

plt.figure(figsize=(10,5))
sns.barplot(data=severity_binned, x='AgeBin', y='HospRate_per10k', color='#0072B2')
plt.title('Hospitalization Rate per 10,000 Cases by Age Group')
plt.xlabel('Age Group (years)')
plt.ylabel('Hospitalizations per 10,000 Cases')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,5))
sns.barplot(data=severity_binned, x='AgeBin', y='DeathRate_per100k', color='#E69F00')
plt.title('Death Rate per 100,000 Cases by Age Group')
plt.xlabel('Age Group (years)')
plt.ylabel('Deaths per 100,000 Cases')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Highest Risk
peak_hosp = severity_binned.loc[severity_binned['HospRate_per10k'].idxmax()]
peak_death = severity_binned.loc[severity_binned['DeathRate_per100k'].idxmax()]

print(
    f"Highest hospitalization rate: {peak_hosp['AgeBin']} "
    f"({peak_hosp['HospRate_per10k']:.1f} per 10,000 cases)"
)

print(
    f"Highest death rate: {peak_death['AgeBin']} "
    f"({peak_death['DeathRate_per100k']:.1f} per 100,000 cases)"
)

### Summary of Severity Patterns

Although case counts during the 2024–2025 influenza season in Japan were extremely high, the hospitalization and death data show that severity was strongly age-dependent.

After aligning the fine-grained case age bands (e.g., 0–5 months, 1 year, 5–9 years)
to the broader hospitalization bins (0–1, 1–9, 10–19, …, 80+), hospitalization rates
rose steeply with age. Children aged 10-14 experienced the highest case volume, but older
adults (especially 70–79 years) had the highest hospitalizations per 10,000 cases.
Deaths were rare overall, but the death rate per 100,000 cases was concentrated in
older age groups, consistent with typical influenza severity patterns.

These findings support the interpretation that the 2024–2025 season was concerning
not because of unusually high lethality, but because of the enormous number of cases,
particularly among children, combined with a concentrated burden of severe outcomes
in older adults.

### Regional Comparison

In [ ]:
# Clean
regional_df = dfs['Detection_H1N1_mH275Y_byp_2425'].copy()
regional_df.columns = regional_df.columns.str.strip()
regional_df.head()

# Antiviral Resistance Rate by Prefecture
regional_df['MutantRate'] = (
    regional_df['Number of H275Y Mutant Viruses'] /
    regional_df['Number of Viruses Tested']
)

regional_df['MixedRate'] = (
    regional_df['Number of 275H/Y Mixed Populations'] /
    regional_df['Number of Viruses Tested']
)

# Prefecture by sampling
regional_totals = (
    regional_df.groupby('Prefecture', as_index=False)
    .agg({
        'Number of Viruses Tested': 'sum',
        'Number of H275Y Mutant Viruses': 'sum',
        'Number of 275H/Y Mixed Populations': 'sum'
    })
    .sort_values('Number of Viruses Tested', ascending=False)
)

regional_totals.head(10)

# Top 10 Prefectures by viruses tested
import seaborn as sns
import matplotlib.pyplot as plt

top10 = regional_totals.head(10)

sns.barplot(
    data=top10,
    x='Number of Viruses Tested',
    y='Prefecture',
    color='#4C72B0'  
)

plt.title("Top 10 Prefectures by Influenza Samples Tested (2024–25 Season)")
plt.xlabel("Viruses Tested")
plt.ylabel("Prefecture")
plt.tight_layout()
plt.show()

# Prefecture with highest antiviral rate
regional_totals['MutantRate'] = (
    regional_totals['Number of H275Y Mutant Viruses'] /
    regional_totals['Number of Viruses Tested']
)

regional_mutations = (
    regional_totals.sort_values('MutantRate', ascending=False)
    .head(10)
)

plt.figure(figsize=(10,6))

sns.barplot(
    data=regional_mutations,
    x='MutantRate',
    y='Prefecture',
    color='#4C72B0'
)

plt.title("Prefectures With Highest H275Y Antiviral Resistance Rate (2024–25)")
plt.xlabel("Mutation Rate")
plt.ylabel("Prefecture")
plt.tight_layout()
plt.show()

# Geo
url = "https://raw.githubusercontent.com/piuccio/open-data-jp-prefectures-geojson/master/output/prefectures.geojson"
urllib.request.urlretrieve(url, "prefectures.geojson")
print("Downloaded prefectures.geojson")

# Match Prefecture Name
en_to_jp = {
    "Hokkaido": "北海道",
    "Aomori": "青森県",
    "Iwate": "岩手県",
    "Miyagi": "宮城県",
    "Akita": "秋田県",
    "Yamagata": "山形県",
    "Fukushima": "福島県",
    "Ibaraki": "茨城県",
    "Tochigi": "栃木県",
    "Gunma": "群馬県",
    "Saitama": "埼玉県",
    "Chiba": "千葉県",
    "Tokyo": "東京都",
    "Kanagawa": "神奈川県",
    "Niigata": "新潟県",
    "Toyama": "富山県",
    "Ishikawa": "石川県",
    "Fukui": "福井県",
    "Yamanashi": "山梨県",
    "Nagano": "長野県",
    "Gifu": "岐阜県",
    "Shizuoka": "静岡県",
    "Aichi": "愛知県",
    "Mie": "三重県",
    "Shiga": "滋賀県",
    "Kyoto": "京都府",
    "Osaka": "大阪府",
    "Hyogo": "兵庫県",
    "Nara": "奈良県",
    "Wakayama": "和歌山県",
    "Tottori": "鳥取県",
    "Shimane": "島根県",
    "Okayama": "岡山県",
    "Hiroshima": "広島県",
    "Yamaguchi": "山口県",
    "Tokushima": "徳島県",
    "Kagawa": "香川県",
    "Ehime": "愛媛県",
    "Kochi": "高知県",
    "Fukuoka": "福岡県",
    "Saga": "佐賀県",
    "Nagasaki": "長崎県",
    "Kumamoto": "熊本県",
    "Oita": "大分県",
    "Miyazaki": "宮崎県",
    "Kagoshima": "鹿児島県",
    "Okinawa": "沖縄県"
}

regional_totals['Prefecture_JP'] = (
    regional_totals['Prefecture']
    .astype(str).str.strip()
    .map(en_to_jp)
)

# Load GeoJSON
with open("prefectures.geojson", "r", encoding="utf-8") as f:
    jp_geo = json.load(f)

# Choropleth
pio.renderers.default = "notebook"  # or "jupyterlab" if that works better

fig = px.choropleth(
    regional_totals,
    geojson=jp_geo,
    locations="Prefecture_JP",          # Japanese names we just created
    featureidkey="properties.P",        # Japanese names in GeoJSON
    color="MutantRate",
    color_continuous_scale="Reds",
    hover_name="Prefecture",
    hover_data={
        "Number of Viruses Tested": True,
        "Number of H275Y Mutant Viruses": True,
        "MutantRate": ':.3f'
    },
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(
    title="H275Y Antiviral Resistance Rate by Prefecture (2024–2025)",
    margin={"r":0, "t":40, "l":0, "b":0},
)
fig.show()

fig2 = px.choropleth(
    regional_totals,
    geojson=jp_geo,
    locations="Prefecture_JP",
    featureidkey="properties.P",
    color="Number of Viruses Tested",
    color_continuous_scale="Blues",
    hover_name="Prefecture",
    hover_data={
        "Number of Viruses Tested": True,
        "MutantRate": ':.3f'
    },
)

fig2.update_geos(fitbounds="locations", visible=False)
fig2.update_layout(
    title="Influenza Sampling Volume by Prefecture (Viruses Tested, 2024–2025)",
    margin={"r":0, "t":40, "l":0, "b":0},
)
fig2.show()


### Regional Comparison

The regional analysis of antiviral surveillance data shows significant geographic variation
in influenza activity during the 2024–2025 season. Tokyo, Yamagata, and Kanagawa recorded
the highest number of viruses tested, reflecting both population density and regional
influenza circulation. Interestingly, Yamagata—despite its smaller population—showed
unusually high testing volume, consistent with historical patterns of earlier influenza
activity in northern Japan.

When examining antiviral resistance, Aomori displayed the highest H275Y mutation rate,
standing out substantially compared to other prefectures. Kanagawa and Saitama also showed
elevated resistance rates, suggesting that resistant strains may have emerged both in
northern prefectures and within the Tokyo metropolitan area. These findings highlight the
importance of regional surveillance to detect localized clusters of antiviral-resistant
influenza strains.

### Rate of Change Analysis

In [ ]:
# Week over week % change
season_trend['WoW_Change'] = season_trend['Count'].pct_change() * 100
season_trend.head()

# Rolling 3 week % change
season_trend['Roll_Change'] = (
    season_trend['Count']
    .pct_change(periods=3) * 100
)

# Plot week over week
plt.figure(figsize=(10,5))
plt.plot(season_trend['SeasonWeek'], season_trend['WoW_Change'], marker='o')
plt.axhline(0, color='black', linewidth=1)

plt.title("Week-over-Week Influenza Case Growth (2024–2025 Season)")
plt.xlabel("Season Week")
plt.ylabel("Percent Change (%)")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Max growth week
peak_growth = season_trend.loc[season_trend['WoW_Change'].idxmax()]
peak_growth

print(
    f"Fastest acceleration occurred in Season Week {int(peak_growth['SeasonWeek'])}: "
    f"{peak_growth['WoW_Change']:.1f}% increase from previous week."
)

# Rolling growth plot
plt.figure(figsize=(10,5))
plt.plot(season_trend['SeasonWeek'], season_trend['Roll_Change'], color='purple', marker='o')

plt.axhline(0, color='black', linewidth=1)
plt.title("3-Week Rolling Percent Change in Influenza Cases")
plt.xlabel("Season Week")
plt.ylabel("3-Week Percent Change (%)")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### Forecasting

## Insights

## Conclusion